In [83]:
import numpy as np, os, sys
from scipy.io import loadmat
import wfdb
import matplotlib

import matplotlib.pyplot as pp


In [84]:
def makealldirectoriesatonce(output_directory,actual_labels,classes):
    lead_names=['I','II','III','aVR','aVL','aVF','V1', 'V2', 'V3', 'V4','V5','V6']
    tmp_class_names=list(actual_labels.values())
    
    
    #print(tmp_class_names)
    #print(actual_labels)
    if not os.path.isdir(output_directory):
        os.mkdir(output_directory)
    
    for i in range(len(tmp_class_names)):
        if not os.path.isdir(output_directory+"/"+str(tmp_class_names[i])):
            os.mkdir(output_directory+"/"+str(tmp_class_names[i]))

In [85]:
def drawandsavefig_temp(PTDETAILSDATA, data, output_directory):
    leads=list()
    ptID=str(PTDETAILSDATA[0])
    num_leads=int(PTDETAILSDATA[1])
    
    
    for i in range(num_leads):
        
        leads.append(PTDETAILSDATA[2*i+3])
        newdir=output_directory+"/"+str(PTDETAILSDATA[-1])
        savingto=newdir+"/"+str(leads[i])+".png"
        
        print("Saving to: "+ savingto)
        
        x=pp.figure(num=None, figsize=(14, 6), dpi=300, facecolor='w', edgecolor='k')
        #pp.axis('off')
        pp.grid(True)
        pp.plot(data[i][:2000])
        if not os.path.exists(savingto):
            pp.savefig(savingto,dpi=300)
        pp.close(x)
        #pp.show()
        

In [86]:
def load_data(filename):
    x = loadmat(filename)
    data = np.asarray(x['val'], dtype=np.float64)

    new_file = filename.replace('.mat','.hea')
    input_header_file = os.path.join(new_file)

    with open(input_header_file,'r') as f:
        header_data=f.readlines()
    return data, header_data












In [87]:
def getactuallabels(classes):
    dictclasses={'164884008':'Premature Ventricular Complexes', '164889003':'Atrial Fibrillation', '164909002':'Left Bundle Branch Block', '164931005':'ST Elevation', '270492004':'1st Degree AV Block', '284470004':'Premature Atrial Contraction', '426783006':'Sinus Rhythm', '429622005':'ST Depression', '59118001':'Right Bundle Branch Block'}
    dictclasses_abrevated={'164884008':'PVC', '164889003':'AF', '164909002':'LBBB', '164931005':'STE', '270492004':'IAVB', '284470004':'PAC', '426783006':'SNR', '429622005':'STD', '59118001':'RBBB'}
    '''for label in classes:
        #print(label)
        print(dictclasses_abrevated[str(label)] + " : " + dictclasses[str(label)])
    '''
    return dictclasses, dictclasses_abrevated

In [88]:
def get_classes(input_directory,files):
    classes=set()
    for f in files:
        g = f.replace('.mat','.hea')
        input_file = os.path.join(input_directory,g)
        with open(input_file,'r') as f:
            for lines in f:
                if lines.startswith('#Dx'):
                    tmp = lines.split(': ')[1].split(',')
                    for c in tmp:
                        classes.add(c.strip())
    return sorted(classes)

In [89]:
def get_12ECG_features(tmp_input_file, data, header_data):

    ptDetails=list()
    gainLeadDetails=list()
    
    tmp_hea = header_data[0].split(' ')
    
    ptID = tmp_hea[0]
    ptDetails.append(ptID)
    num_leads = int(tmp_hea[1])
    ptDetails.append(num_leads)
    sample_Fs= int(tmp_hea[2])
    ptDetails.append(sample_Fs)
    gain_lead = np.zeros(num_leads)
    #ptDetails.append(gain_lead)
    for ii in range(num_leads):
        tmp_hea = header_data[ii+1].split(' ')
        gain_lead[ii] = int(tmp_hea[2].split('/')[0])
        lead_no= str(tmp_hea[8]).strip()
        ptDetails.append(lead_no)
        ptDetails.append(gain_lead[ii])
        
    # for testing, we included the mean age of 57 if the age is a NaN
    # This value will change as more data is being released
    for iline in header_data:
        if iline.startswith('#Age'):
            tmp_age = iline.split(': ')[1].strip()
            age = int(tmp_age if tmp_age != 'NaN' else 57)
            ptDetails.append(age)
        elif iline.startswith('#Sex'):
            tmp_sex = iline.split(': ')[1]
            if tmp_sex.strip()=='Female':
                sex=1
            else:
                sex=0
            ptDetails.append(sex)
        elif iline.startswith('#Dx'):
            label = iline.split(': ')[1].split(',')[0].strip()
            ptDetails.append(label)
            
            
            #ptDetails.append()
    #ptDetails.append(tmp_input_file, ptID,label,num_leads,sample_Fs, gain_lead,age,sex,gainLeadDetails)
    return ptDetails


In [90]:
def main():
    
    
    DRAW_SAVE_ONLY_IMPORTANT_PICS=1
    TO_DRAW_PICS=list()
    DRAWN_PICS=list()
    
    
    input_directory="../DATA/Training_WFDB"
    output_directory="../DATA/Images_For_Thesis"
    
    
    #data=readfile("AOOO.mat")
    #header_data=readfile("A0000.hea")
    input_files = []
    
    
    for f in os.listdir(input_directory):
        if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('mat'):
            input_files.append(f)
    
    
    
    
    num_files = len(input_files)
    
    
    #classes=get_classes(input_directory,input_files)
    classes= ['164884008', '164889003', '164909002', '164931005', '270492004', '284470004', '426783006', '429622005', '59118001']
    print(classes)
    actual_label=getactuallabels(classes)
    print(actual_label[0])
    
    makealldirectoriesatonce(output_directory,actual_label[1],classes)
    '''for label in classes:
        #actual label
        print(actual_label[0][str(label)])
        #abrevated label
        print(actual_label[1][str(label)])
    '''
    
    
    for i, f in enumerate(input_files):
        #print('    {}/{}...'.format(i+1, num_files))
        tmp_input_file = os.path.join(input_directory,f)
        
        
        data,header_data = load_data(tmp_input_file)
        PTDETAILSDATA=get_12ECG_features(tmp_input_file, data, header_data)
        PTDETAILSDATA.append(actual_label[0][str(PTDETAILSDATA[-1])])
        PTDETAILSDATA.append(actual_label[1][str(PTDETAILSDATA[-2])])
        
        
        #print(PTDETAILSDATA)
        drawandsavefig_temp(PTDETAILSDATA,data,output_directory)
        
        '''if DRAW_SAVE_ONLY_IMPORTANT_PICS==1:
            if len(TO_DRAW_PICS) > 0:
                drawandsavefig_temp(PTDETAILSDATA,data,output_directory, )
        '''
        
        #print(header_data)
        
        
        #annotation = wfdb.rdann(tmp_input_file.replace('.hea',''), 'atr', sampto=3000)
        #wfdb.plotrec(record, annotation = " ",title='Record 100 from MIT-BIH Arrhythmia Database',timeunits = 'seconds', figsize = (10,4), ecggrids = 'all')
        
        if i==100:
            break

In [91]:
if __name__=="__main__":
    main()

['164884008', '164889003', '164909002', '164931005', '270492004', '284470004', '426783006', '429622005', '59118001']
{'164884008': 'Premature Ventricular Complexes', '164889003': 'Atrial Fibrillation', '164909002': 'Left Bundle Branch Block', '164931005': 'ST Elevation', '270492004': '1st Degree AV Block', '284470004': 'Premature Atrial Contraction', '426783006': 'Sinus Rhythm', '429622005': 'ST Depression', '59118001': 'Right Bundle Branch Block'}
Saving to: ../DATA/Images_For_Thesis/RBBB/I.png
Saving to: ../DATA/Images_For_Thesis/RBBB/II.png
Saving to: ../DATA/Images_For_Thesis/RBBB/III.png
Saving to: ../DATA/Images_For_Thesis/RBBB/aVR.png
Saving to: ../DATA/Images_For_Thesis/RBBB/aVL.png
Saving to: ../DATA/Images_For_Thesis/RBBB/aVF.png
Saving to: ../DATA/Images_For_Thesis/RBBB/V1.png
Saving to: ../DATA/Images_For_Thesis/RBBB/V2.png
Saving to: ../DATA/Images_For_Thesis/RBBB/V3.png
Saving to: ../DATA/Images_For_Thesis/RBBB/V4.png
Saving to: ../DATA/Images_For_Thesis/RBBB/V5.png
Savin

Saving to: ../DATA/Images_For_Thesis/STD/V1.png
Saving to: ../DATA/Images_For_Thesis/STD/V2.png
Saving to: ../DATA/Images_For_Thesis/STD/V3.png
Saving to: ../DATA/Images_For_Thesis/STD/V4.png
Saving to: ../DATA/Images_For_Thesis/STD/V5.png
Saving to: ../DATA/Images_For_Thesis/STD/V6.png
Saving to: ../DATA/Images_For_Thesis/RBBB/I.png
Saving to: ../DATA/Images_For_Thesis/RBBB/II.png
Saving to: ../DATA/Images_For_Thesis/RBBB/III.png
Saving to: ../DATA/Images_For_Thesis/RBBB/aVR.png
Saving to: ../DATA/Images_For_Thesis/RBBB/aVL.png
Saving to: ../DATA/Images_For_Thesis/RBBB/aVF.png
Saving to: ../DATA/Images_For_Thesis/RBBB/V1.png
Saving to: ../DATA/Images_For_Thesis/RBBB/V2.png
Saving to: ../DATA/Images_For_Thesis/RBBB/V3.png
Saving to: ../DATA/Images_For_Thesis/RBBB/V4.png
Saving to: ../DATA/Images_For_Thesis/RBBB/V5.png
Saving to: ../DATA/Images_For_Thesis/RBBB/V6.png
Saving to: ../DATA/Images_For_Thesis/SNR/I.png
Saving to: ../DATA/Images_For_Thesis/SNR/II.png
Saving to: ../DATA/Images_

Saving to: ../DATA/Images_For_Thesis/RBBB/V5.png
Saving to: ../DATA/Images_For_Thesis/RBBB/V6.png
Saving to: ../DATA/Images_For_Thesis/SNR/I.png
Saving to: ../DATA/Images_For_Thesis/SNR/II.png
Saving to: ../DATA/Images_For_Thesis/SNR/III.png
Saving to: ../DATA/Images_For_Thesis/SNR/aVR.png
Saving to: ../DATA/Images_For_Thesis/SNR/aVL.png
Saving to: ../DATA/Images_For_Thesis/SNR/aVF.png
Saving to: ../DATA/Images_For_Thesis/SNR/V1.png
Saving to: ../DATA/Images_For_Thesis/SNR/V2.png
Saving to: ../DATA/Images_For_Thesis/SNR/V3.png
Saving to: ../DATA/Images_For_Thesis/SNR/V4.png
Saving to: ../DATA/Images_For_Thesis/SNR/V5.png
Saving to: ../DATA/Images_For_Thesis/SNR/V6.png
Saving to: ../DATA/Images_For_Thesis/SNR/I.png
Saving to: ../DATA/Images_For_Thesis/SNR/II.png
Saving to: ../DATA/Images_For_Thesis/SNR/III.png
Saving to: ../DATA/Images_For_Thesis/SNR/aVR.png
Saving to: ../DATA/Images_For_Thesis/SNR/aVL.png
Saving to: ../DATA/Images_For_Thesis/SNR/aVF.png
Saving to: ../DATA/Images_For_Th

Saving to: ../DATA/Images_For_Thesis/AF/aVL.png
Saving to: ../DATA/Images_For_Thesis/AF/aVF.png
Saving to: ../DATA/Images_For_Thesis/AF/V1.png
Saving to: ../DATA/Images_For_Thesis/AF/V2.png
Saving to: ../DATA/Images_For_Thesis/AF/V3.png
Saving to: ../DATA/Images_For_Thesis/AF/V4.png
Saving to: ../DATA/Images_For_Thesis/AF/V5.png
Saving to: ../DATA/Images_For_Thesis/AF/V6.png
Saving to: ../DATA/Images_For_Thesis/LBBB/I.png
Saving to: ../DATA/Images_For_Thesis/LBBB/II.png
Saving to: ../DATA/Images_For_Thesis/LBBB/III.png
Saving to: ../DATA/Images_For_Thesis/LBBB/aVR.png
Saving to: ../DATA/Images_For_Thesis/LBBB/aVL.png
Saving to: ../DATA/Images_For_Thesis/LBBB/aVF.png
Saving to: ../DATA/Images_For_Thesis/LBBB/V1.png
Saving to: ../DATA/Images_For_Thesis/LBBB/V2.png
Saving to: ../DATA/Images_For_Thesis/LBBB/V3.png
Saving to: ../DATA/Images_For_Thesis/LBBB/V4.png
Saving to: ../DATA/Images_For_Thesis/LBBB/V5.png
Saving to: ../DATA/Images_For_Thesis/LBBB/V6.png
Saving to: ../DATA/Images_For_T

Saving to: ../DATA/Images_For_Thesis/LBBB/V2.png
Saving to: ../DATA/Images_For_Thesis/LBBB/V3.png
Saving to: ../DATA/Images_For_Thesis/LBBB/V4.png
Saving to: ../DATA/Images_For_Thesis/LBBB/V5.png
Saving to: ../DATA/Images_For_Thesis/LBBB/V6.png
Saving to: ../DATA/Images_For_Thesis/STE/I.png
Saving to: ../DATA/Images_For_Thesis/STE/II.png
Saving to: ../DATA/Images_For_Thesis/STE/III.png
Saving to: ../DATA/Images_For_Thesis/STE/aVR.png
Saving to: ../DATA/Images_For_Thesis/STE/aVL.png
Saving to: ../DATA/Images_For_Thesis/STE/aVF.png
Saving to: ../DATA/Images_For_Thesis/STE/V1.png
Saving to: ../DATA/Images_For_Thesis/STE/V2.png
Saving to: ../DATA/Images_For_Thesis/STE/V3.png
Saving to: ../DATA/Images_For_Thesis/STE/V4.png
Saving to: ../DATA/Images_For_Thesis/STE/V5.png
Saving to: ../DATA/Images_For_Thesis/STE/V6.png
Saving to: ../DATA/Images_For_Thesis/SNR/I.png
Saving to: ../DATA/Images_For_Thesis/SNR/II.png
Saving to: ../DATA/Images_For_Thesis/SNR/III.png
Saving to: ../DATA/Images_For_Th

Saving to: ../DATA/Images_For_Thesis/AF/V5.png
Saving to: ../DATA/Images_For_Thesis/AF/V6.png
Saving to: ../DATA/Images_For_Thesis/STE/I.png
Saving to: ../DATA/Images_For_Thesis/STE/II.png
Saving to: ../DATA/Images_For_Thesis/STE/III.png
Saving to: ../DATA/Images_For_Thesis/STE/aVR.png
Saving to: ../DATA/Images_For_Thesis/STE/aVL.png
Saving to: ../DATA/Images_For_Thesis/STE/aVF.png
Saving to: ../DATA/Images_For_Thesis/STE/V1.png
Saving to: ../DATA/Images_For_Thesis/STE/V2.png
Saving to: ../DATA/Images_For_Thesis/STE/V3.png
Saving to: ../DATA/Images_For_Thesis/STE/V4.png
Saving to: ../DATA/Images_For_Thesis/STE/V5.png
Saving to: ../DATA/Images_For_Thesis/STE/V6.png
Saving to: ../DATA/Images_For_Thesis/SNR/I.png
Saving to: ../DATA/Images_For_Thesis/SNR/II.png
Saving to: ../DATA/Images_For_Thesis/SNR/III.png
Saving to: ../DATA/Images_For_Thesis/SNR/aVR.png
Saving to: ../DATA/Images_For_Thesis/SNR/aVL.png
Saving to: ../DATA/Images_For_Thesis/SNR/aVF.png
Saving to: ../DATA/Images_For_Thesis

Saving to: ../DATA/Images_For_Thesis/AF/V1.png
Saving to: ../DATA/Images_For_Thesis/AF/V2.png
Saving to: ../DATA/Images_For_Thesis/AF/V3.png
Saving to: ../DATA/Images_For_Thesis/AF/V4.png
Saving to: ../DATA/Images_For_Thesis/AF/V5.png
Saving to: ../DATA/Images_For_Thesis/AF/V6.png
Saving to: ../DATA/Images_For_Thesis/RBBB/I.png
Saving to: ../DATA/Images_For_Thesis/RBBB/II.png
Saving to: ../DATA/Images_For_Thesis/RBBB/III.png
Saving to: ../DATA/Images_For_Thesis/RBBB/aVR.png
Saving to: ../DATA/Images_For_Thesis/RBBB/aVL.png
Saving to: ../DATA/Images_For_Thesis/RBBB/aVF.png
Saving to: ../DATA/Images_For_Thesis/RBBB/V1.png
Saving to: ../DATA/Images_For_Thesis/RBBB/V2.png
Saving to: ../DATA/Images_For_Thesis/RBBB/V3.png
Saving to: ../DATA/Images_For_Thesis/RBBB/V4.png
Saving to: ../DATA/Images_For_Thesis/RBBB/V5.png
Saving to: ../DATA/Images_For_Thesis/RBBB/V6.png
Saving to: ../DATA/Images_For_Thesis/PVC/I.png
Saving to: ../DATA/Images_For_Thesis/PVC/II.png
Saving to: ../DATA/Images_For_Th

Saving to: ../DATA/Images_For_Thesis/STD/V5.png
Saving to: ../DATA/Images_For_Thesis/STD/V6.png
Saving to: ../DATA/Images_For_Thesis/AF/I.png
Saving to: ../DATA/Images_For_Thesis/AF/II.png
Saving to: ../DATA/Images_For_Thesis/AF/III.png
Saving to: ../DATA/Images_For_Thesis/AF/aVR.png
Saving to: ../DATA/Images_For_Thesis/AF/aVL.png
Saving to: ../DATA/Images_For_Thesis/AF/aVF.png
Saving to: ../DATA/Images_For_Thesis/AF/V1.png
Saving to: ../DATA/Images_For_Thesis/AF/V2.png
Saving to: ../DATA/Images_For_Thesis/AF/V3.png
Saving to: ../DATA/Images_For_Thesis/AF/V4.png
Saving to: ../DATA/Images_For_Thesis/AF/V5.png
Saving to: ../DATA/Images_For_Thesis/AF/V6.png
